# Log 

* Written by HongYongGi / email : yonggi@tesser.co.kr

* Written date : 20230524

---

# Code description

* Read the header file & csv information

* 


---


# Code Flow

* Load CT data

* Tumor coordinate load

* Draw the tumor segmentation box 

* Save the segmentation box

---

# Package Import

In [1]:
# data science
import os, glob, shutil
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Medical Image package
import nibabel as nib
from nibabel.affines import apply_affine
import pydicom
import skimage.io as io

# etc
from tqdm import tqdm
import ipywidgets as widgets
from datetime import date


---
# Utils function

In [2]:
def load_nii(file_path):
    """
    load nii file
    
    Args:
        file_path (str): nii file path
    Returns : 
        nii (nibabel.nifti1.Nifti1Image): NIFTI file
        affine (numpy.ndarray): affine matrix
        header (dict): header information
    
    """
    nii = nib.load(file_path)
    header = nii.header
    affine = nii.affine
    nii = nii.get_fdata()
    
    return nii, affine, header

def read_header(path):
    """
    Reads pixel info from header

    Args:
        path (str): header file path
    Returns:
        pixel_info (str): pixel info
    """
    header = open(path,"r")
    while True:
        line = header.readline()
        print(line)
        if "ElementSpacing = " in line :
            info = line[line.find("ElementSpacing = ")+17:line.find('\n')]
            pixel_info = info.split(' ')
        
        if not line:
            break
    return pixel_info
        
    
def draw_seg(arr, x_tup, y_tup,z_tup):
    """
    
    Args : 
        arr (array): CT image array
        x_tup (tuple): tuple of min and max x coordinates
        y_tup (tuple): tuple of min and max y coordinates
        z_tup (tuple): tuple of min and max z coordinates
        
    Returns : 
        draw_array(array) : Draw CT image with segmentation array
    
    """
    
    segmentation_array = np.matrix.copy(arr)
    
    # x1, x2 = int(x_coor - radius), int(x_coor + radius)
    # y1, y2 = int(y_coor - radius), int(y_coor + radius)
    # z1, z2 = int(z_coor - radius), int(z_coor + radius) # 더 멀리 띄워서 잘 보이게? 지금 상황이면 가장 윗면은 하얀 사각형으로 보일것.
    

    x1, x2 = x_tup[0], x_tup[1]
    y1, y2 = y_tup[0], y_tup[1]
    z1, z2 = z_tup[0], z_tup[1]

    whitest = arr.max()
    
    segmentation_array[x1:x2, y1, z1:z2] = whitest
    segmentation_array[x1:x2, y2, z1:z2] = whitest
    segmentation_array[x1, y1:y2, z1:z2] = whitest
    segmentation_array[x2, y1:y2, z1:z2] = whitest

    return segmentation_array
    
    
def read_info(subject_id , data_frame):
    
    """
    _summary_
    """
    
    x_coord_list = list(data_frame[data_frame['seriesuid'] == subject_id]['coordX'])
    y_coord_list = list(data_frame[data_frame['seriesuid'] == subject_id]['coordY'])
    z_coord_list = list(data_frame[data_frame['seriesuid'] == subject_id]['coordZ'])
    radius_list  = list(data_frame[data_frame['seriesuid'] == subject_id]['diameter_mm'])
    
    return x_coord_list, y_coord_list, z_coord_list, radius_list

# Data Load

In [3]:
data_dir = '/mnt/tesser_nas2/lg_vm_dicom_LUNA16/lg_tm_LUNA16/imagesTr/'
csv_path = '/mnt/tesser_nas2/lg_vm_dicom_LUNA16/annotations.csv'

header_dir = '/mnt/tesser_nas2/lg_vm_dicom_LUNA16/LUNA16_CT/'
nii_file_paths = glob.glob(data_dir + '*.nii.gz')
nii_file_paths.sort()

meta_info = pd.read_csv(csv_path)
print(meta_info)


                                              seriesuid      coordX   
0     1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222... -128.699421  \
1     1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...  103.783651   
2     1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...   69.639017   
3     1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...  -24.013824   
4     1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...    2.441547   
...                                                 ...         ...   
1181  1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950... -160.856298   
1182  1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950... -102.189570   
1183  1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950...  -37.535409   
1184  1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...   43.196112   
1185  1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...  -21.958478   

          coordY      coordZ  diameter_mm  
0    -175.319272 -298.387506     5.651471  
1    -211.925149 -227.121250     4.224708  
2    -140.94458

In [9]:


def printer(idx):

    file_id = os.path.basename(nii_file_paths[idx])[:-7]
    header_path = os.path.join(header_dir, file_id+'.mhd')
    header = open(header_path,"r")
    
    x_coord_list, y_coord_list, z_coord_list, radius_list = read_info(file_id, meta_info)
    
    world_coord_list = [] # list of tuples (min, max)
    for ind in range(len(x_coord_list)):
        coord_list = []

        tup_x = (x_coord_list[ind] - radius_list[ind], x_coord_list[ind] + radius_list[ind])
        tup_y = (y_coord_list[ind] - radius_list[ind], y_coord_list[ind] + radius_list[ind])
        tup_z = (z_coord_list[ind] - radius_list[ind], z_coord_list[ind] + radius_list[ind])
        coord_list.extend([tup_x, tup_y, tup_z])
        world_coord_list.append(coord_list)

    nii, affine, header = load_nii(nii_file_paths[idx])

    affine_inv = np.linalg.inv(affine)

    voxel_coord = []
    for ind in range(len(x_coord_list)):
        coord_list = world_coord_list[ind]
        voxel_xmin, voxel_ymin, voxel_zmin = apply_affine(affine_inv, [-coord_list[0][0], -coord_list[1][0], coord_list[2][0]]) # draw_seg와 함께 real world coord의 박스 경계를 먼저 구해야 한다.
        voxel_xmax, voxel_ymax, voxel_zmax = apply_affine(affine_inv, [-coord_list[0][1], -coord_list[1][1], coord_list[2][1]])
        
        voxel_xmin = int(voxel_xmin)
        voxel_xmax = int(voxel_xmax)
        voxel_ymin = int(voxel_ymin)
        voxel_ymax = int(voxel_ymax)
        voxel_zmin = int(voxel_zmin)
        voxel_zmax = int(voxel_zmax)

        new_coords = [(voxel_xmin, voxel_xmax), (voxel_ymin, voxel_ymax), (voxel_zmin, voxel_zmax)]
        voxel_coord.append(new_coords)

    seg = nii
    for ind in range(len(voxel_coord)):
        this_voxel = voxel_coord[ind]
        seg = draw_seg(seg, this_voxel[0], this_voxel[1], this_voxel[2])
    
    ni_img = nib.Nifti1Image(seg, affine=affine)
    nib.save(ni_img, os.path.join("/mnt/tesser_nas2/lg_vm_dicom_LUNA16/LUNA16_CT_bbox/", file_id + "_bbox.nii.gz"))


# 모든 파일에 대해 실행

# for ind in range( 0, 885):
#     printer(ind)



    

KeyboardInterrupt: 

In [7]:
# file_id = os.path.basename(nii_file_paths[idx])[:-7]
# header_path = os.path.join(header_dir, file_id+'.mhd')
# header = open(header_path,"r")

# file_id


True
1
Tesla P40


In [97]:
# Pixel_information = read_header(header_path)
# Pixel_information

ObjectType = Image

NDims = 3

BinaryData = True

BinaryDataByteOrderMSB = False

CompressedData = False

TransformMatrix = 1 0 0 0 1 0 0 0 1

Offset = -208.800003 -200 -376.75

CenterOfRotation = 0 0 0

AnatomicalOrientation = RAI

ElementSpacing = 0.78125 0.78125 2.5

DimSize = 512 512 131

ElementType = MET_SHORT

ElementDataFile = 1.3.6.1.4.1.14519.5.2.1.6279.6001.128881800399702510818644205032.raw




['0.78125', '0.78125', '2.5']

In [98]:
# x_coord_list, y_coord_list, z_coord_list, radius_list = read_info(file_id, meta_info)


In [103]:
# world_coord_list = [] # list of tuples (min, max)
# for ind in range(len(x_coord_list)):
#     coord_list = []

#     tup_x = (x_coord_list[ind] - radius_list[ind], x_coord_list[ind] + radius_list[ind])
#     tup_y = (y_coord_list[ind] - radius_list[ind], y_coord_list[ind] + radius_list[ind])
#     tup_z = (z_coord_list[ind] - radius_list[ind], z_coord_list[ind] + radius_list[ind])
#     coord_list.extend([tup_x, tup_y, tup_z])
#     world_coord_list.append(coord_list)

# world_coord_list



[[(-112.17775827, -87.53928547000001),
  (25.114814619999997, 49.75328742),
  (-182.7758452, -158.1373724)],
 [(-91.69227878, -65.64199404),
  (53.39701658, 79.44730132),
  (-181.89682077, -155.84653603)]]

real world coord가 x,y 좌표에 -가 붙어서 output.

luna16에서 주는 real world coord에는 -를 붙인 후, affine으로 voxel를 계산하면 된다.

또는, luna16의 real world coord에서 offset을 뺀 후, pixel spacing으로 나눈다.